* https://stackoverflow.com/questions/42471523/how-can-i-generate-a-proper-mnist-image
* https://stackoverflow.com/questions/45539289/convert-image-from-28-28-4-to-2d-flat-array-and-write-to-csv
* https://stackoverflow.com/questions/61552402/if-image-has-28-28-3-shape-how-do-i-convert-it-to-28-28-1
* https://stackoverflow.com/questions/51205502/convert-a-black-and-white-image-to-array-of-numbers

In [1]:
! conda install xgboost -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/studio-lab-user/.conda/envs/default

  added / updated specs:
    - xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    ca-certificates-2022.5.18.1|       ha878542_0         144 KB  conda-forge
    certifi-2022.5.18.1        |   py39hf3d152e_0         150 KB  conda-forge
    libxgboost-1.5.1           |   cpu_h3d145d1_2         3.6 MB  conda-forge
    py-xgboost-1.5.1           |cpu_py39h4655687_2         152 KB  conda-forge
    xgboost-1.5.1              |cpu_py39h4655687_2          12 KB  conda-forge
    --------------------------------------------------------

In [24]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import numpy as np
import pandas as pd
import pickle
from sklearn.datasets import fetch_openml

In [3]:
# import the mnist datsaet

mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [4]:
# separate features and target
X, y = mnist["data"], mnist["target"]

In [5]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                       test_size=0.2, 
                                       random_state=42)

In [6]:
# standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Single Decision Tree

In [20]:
# instantiate with arbitrary hyperparameters
tree_model = DecisionTreeClassifier(max_depth=7, 
                               criterion='entropy', 
                               min_samples_leaf=10,
                               class_weight='balanced')

In [21]:
# train the model
tree_model.fit(X_train_scaled, y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=7, min_samples_leaf=10)

In [22]:
# predict
y_preds=tree_model.predict(X_test_scaled)
print(list(y_preds[:10]))
print(list(y_test[:10]))

['8', '4', '9', '7', '7', '0', '2', '2', '7', '4']
['8', '4', '8', '7', '7', '0', '6', '2', '7', '4']


In [23]:
# evaluate
print('Accuracy:', metrics.accuracy_score(y_test, y_preds))
print('Precision:', metrics.precision_score(y_test, y_preds,average='macro'))
print('Recall:', metrics.recall_score(y_test, y_preds,average='macro'))
print('F1 Score:', metrics.f1_score(y_test, y_preds,average='macro'))

Accuracy: 0.7998571428571428
Precision: 0.8008533054956182
Recall: 0.7981828893750074
F1 Score: 0.7983136439521091


## Random Forest

In [7]:
# modeling: random forest (arbitrary hyperparameters)
rf_model = RandomForestClassifier(max_depth=8, min_samples_leaf=10, n_estimators=100)
rf_model.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=8, min_samples_leaf=10)

In [8]:
# predict
y_preds=rf_model.predict(X_test_scaled)
print(list(y_preds[:10]))
print(list(y_test[:10]))

['8', '4', '8', '7', '7', '0', '6', '2', '7', '9']
['8', '4', '8', '7', '7', '0', '6', '2', '7', '4']


In [9]:
# evaluate
print('Accuracy:', metrics.accuracy_score(y_test, y_preds))
print('Precision:', metrics.precision_score(y_test, y_preds,average='macro'))
print('Recall:', metrics.recall_score(y_test, y_preds,average='macro'))
print('F1 Score:', metrics.f1_score(y_test, y_preds,average='macro'))

0.9243571428571429
0.925164577445767
0.9233458790759874
0.9239505976803575


## XG Boost

There are in general two ways that you can control overfitting in XGBoost:

- The first way is to directly control model complexity.

    - This includes max_depth, min_child_weight and gamma.

- The second way is to add randomness to make training robust to noise.

    - This includes subsample and colsample_bytree.

    - You can also reduce stepsize eta. Remember to increase num_round when you do so.

[source](https://xgboost.readthedocs.io/en/stable/tutorials/param_tuning.html#:~:text=There%20are%20in,you%20do%20so.)

In [10]:
# modeling: random forest (arbitrary hyperparameters)
xgb_model = XGBClassifier(max_depth=6, min_child_weight=1, gamma=0, subsample=1, learning_rate=0.3)
xgb_model.fit(X_train_scaled, y_train)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:59:02] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.3, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [11]:
# predict
y_preds=xgb_model.predict(X_test_scaled)
print(list(y_preds[:10]))
print(list(y_test[:10]))

['8', '4', '8', '7', '7', '0', '6', '2', '7', '4']
['8', '4', '8', '7', '7', '0', '6', '2', '7', '4']


In [12]:
# evaluate
print('Accuracy:', metrics.accuracy_score(y_test, y_preds))
print('Precision:', metrics.precision_score(y_test, y_preds,average='macro'))
print('Recall:', metrics.recall_score(y_test, y_preds,average='macro'))
print('F1 Score:', metrics.f1_score(y_test, y_preds,average='macro'))

0.978
0.9779107763913576
0.9780370058804954
0.9779629903443656


## Evaluate on new data

In [30]:
# this is an example of 28x28 input from a user-created handwritten digit
some_digit = pd.read_csv('some_digit.csv')
some_digit.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# convert to expected format

In [31]:
## read in our pickle file
filename = open('user-input-digit.pkl', 'rb')
unpickled_digit = pickle.load(filename)
filename.close()

In [32]:
# show the digit
unpickled_digit.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,5,36,36,36,...,36,20,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,213,255,255,255,...,255,255,36,0,0,0,0,0,0,0


## Pickle the trained models

In [27]:

# tree
f = open('tree_model.pkl', 'wb')
pickle.dump(tree_model, f)
f.close()  

In [ ]:
# random forest
f = open('rf_model.pkl', 'wb')
pickle.dump(rf_model, f)
f.close()  

In [ ]:
# XG Boost
f = open('xgb_model.pkl', 'wb')
pickle.dump(xgb_model, f)
f.close()  